# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill</span>

## <span style='color:#2656a3'> 🗒️ The notebook is divided into the following sections:
1. Loading data and process features
2. Connecting to Hopsworks Feature Store
3. Creating feature groups and uploading them to the feature store

## <span style='color:#2656a3'> ⚙️ Import of Libraries and Packages

First, we install the Python packages required for this notebook. We'll use the --quiet command after specifying the names of the libraries to ensure a silent installation process. Then, we proceed to import all the necessary libraries.

In [1]:
# Install of the packages for hopsworks
# !pip install -U hopsworks --quiet

In [2]:
# First we go one back in our directory to access the folder with our functions
%cd ..

# Now we import the functions from the features folder
# This is the functions we have created to generate features for electricity prices and weather measures
from features import electricity_prices, weather_measures, calendar

# We go back into the notebooks folder
%cd notebooks

/Users/tobiasmjensen/Documents/aau_bds/m5_data-engineering-and-mlops/exam_assigment/MLOPs-Assignment-
/Users/tobiasmjensen/Documents/aau_bds/m5_data-engineering-and-mlops/exam_assigment/MLOPs-Assignment-/notebooks


In [3]:
# Importing the packages for the needed libraries for the Jupyter notebook
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

# Ignore warnings
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

# <span style="color:#2656a3;"> 🤖 Transformation Functions</span>

We preprocess our data using *min-max scaling* on the numerical features and *label encoding* on the one categorical feature we have.
To achieve this, we create a mapping between our features and transformation functions. This ensures that transformation functions like min-max scaling are applied exclusively on the training data, preventing any data leakage into the validation or test sets.

To achieve this, we create a mapping between our features and transformation functions - ved ikke om man kan sige det her?

## <span style="color:#2656a3;"> 💽 Load the Historical Data

The data used comes from different sources:

- Electricity prices in Denmark on hourly basis per day from [Energinet](https://www.energidataservice.dk). Loacated in the *featuresfolder* under electricity_prices.
- Different meteorological observations based on Aalborg Denmark from [Open Meteo](https://www.open-meteo.com). Loacated in the *featuresfolder* under weather_measures.
- Danish calendar that categorizes dates into types based on whether it is a weekday or not. This files is made manually by the group and is located in the *datafolder* inside this repository.
- Weather Forecast based on Aalborg Denmark from [Open Meteo](https://www.open-meteo.com). Loacated in the *featuresfolder* under weather_measures. (This data is used later to parse in new real-time weather data)


### <span style="color:#2656a3;">💸 Electricity prices per day from Energinet
This first dataset is Electricity prices on hourly basis per day from Energinet/Dataservice.

In [4]:
# Fetching historical electricity prices for area DK1 from January 1, 2022
# Note: The end date is currently left out to retrieve data up to the present date of yesterday 
# Today is not included in the data as it is not historical data
electricity_df = electricity_prices.electricity_prices(
    historical=True, 
    area=["DK1"], 
    start='2022-01-01', 
    #end='2023-12-31'
)

In [5]:
# Display the first 5 rows of the electricity dataframe
electricity_df.head(5)

,timestamp,datetime,date,hour,dk1_spotpricedkk_kwh
0,1640995200000,2022-01-01 00:00:00,2022-01-01,0,0.37220
1,1640998800000,2022-01-01 01:00:00,2022-01-01,1,0.30735
2,1641002400000,2022-01-01 02:00:00,2022-01-01,2,0.32141
3,1641006000000,2022-01-01 03:00:00,2022-01-01,3,0.33806
4,1641009600000,2022-01-01 04:00:00,2022-01-01,4,0.28013


In [6]:
# Display the last 5 rows of the electricity dataframe
electricity_df.tail(5)

,timestamp,datetime,date,hour,dk1_spotpricedkk_kwh
20464,1714676400000,2024-05-02 19:00:00,2024-05-02,19,0.31266
20465,1714680000000,2024-05-02 20:00:00,2024-05-02,20,0.31318
20466,1714683600000,2024-05-02 21:00:00,2024-05-02,21,0.31266
20467,1714687200000,2024-05-02 22:00:00,2024-05-02,22,0.28245
20468,1714690800000,2024-05-02 23:00:00,2024-05-02,23,0.25306


In [7]:
# Showing the information for the electricity dataframe
electricity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20469 entries, 0 to 20468
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             20469 non-null  int64         
 1   datetime              20469 non-null  datetime64[ns]
 2   date                  20469 non-null  object        
 3   hour                  20469 non-null  int64         
 4   dk1_spotpricedkk_kwh  20469 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 799.7+ KB


### <span style="color:#2656a3;"> 🌤 Weather measurements from Open Meteo
Next weather measurements from Open Meteo is fetched.

#### <span style="color:#2656a3;"> 🕰️ Historical Weather Measures

In [8]:
# Fetching historical weather measurements from January 1, 2022
# Note: The end date is currently left out to retrieve data up to the present date of yesterday 
# Today is not included in the data as it is not historical data
historical_weather_df = weather_measures.historical_weather_measures(
    historical=True, 
    start = '2022-01-01', 
    #end = '2023-12-31'
)

In [9]:
# Display the first 5 rows of the weather dataframe
historical_weather_df.head(5)

,timestamp,datetime,date,hour,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1640995200000,2022-01-01 00:00:00,2022-01-01,0,6.7,100.0,0.0,0.0,0.0,3.0,100.0,16.2,36.0
1,1640998800000,2022-01-01 01:00:00,2022-01-01,1,6.6,100.0,0.0,0.0,0.0,3.0,100.0,16.2,30.2
2,1641002400000,2022-01-01 02:00:00,2022-01-01,2,6.7,99.0,0.0,0.0,0.0,3.0,100.0,15.5,30.6
3,1641006000000,2022-01-01 03:00:00,2022-01-01,3,6.7,100.0,0.0,0.0,0.0,3.0,100.0,12.7,28.8
4,1641009600000,2022-01-01 04:00:00,2022-01-01,4,6.7,99.0,0.0,0.0,0.0,3.0,100.0,10.6,23.8


In [10]:
# Display the last 5 rows of the weather dataframe
historical_weather_df.tail(5)

,timestamp,datetime,date,hour,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
20443,1714590000000,2024-05-01 19:00:00,2024-05-01,19,15.4,71.0,0.0,0.0,0.0,0.0,13.0,14.8,25.9
20444,1714593600000,2024-05-01 20:00:00,2024-05-01,20,14.7,69.0,0.0,0.0,0.0,0.0,7.0,19.2,33.8
20445,1714597200000,2024-05-01 21:00:00,2024-05-01,21,14.3,67.0,0.0,0.0,0.0,0.0,2.0,20.5,37.4
20446,1714600800000,2024-05-01 22:00:00,2024-05-01,22,13.4,68.0,0.0,0.0,0.0,0.0,7.0,20.2,37.4
20447,1714604400000,2024-05-01 23:00:00,2024-05-01,23,12.4,70.0,0.0,0.0,0.0,0.0,17.0,18.8,36.4


In [11]:
# Showing the information for the weather dataframe
historical_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20448 entries, 0 to 20447
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             20448 non-null  int64         
 1   datetime              20448 non-null  datetime64[ns]
 2   date                  20448 non-null  object        
 3   hour                  20448 non-null  int64         
 4   temperature_2m        20448 non-null  float64       
 5   relative_humidity_2m  20448 non-null  float64       
 6   precipitation         20448 non-null  float64       
 7   rain                  20448 non-null  float64       
 8   snowfall              20448 non-null  float64       
 9   weather_code          20448 non-null  float64       
 10  cloud_cover           20448 non-null  float64       
 11  wind_speed_10m        20448 non-null  float64       
 12  wind_gusts_10m        20448 non-null  float64       
dtypes: datetime64[ns

#### <span style="color:#2656a3;"> 🌈  Forecast Weather Measures
Weather Forecast from Open Meteo is now being fetched. This data is used in part 02 the feature_pipeline to parse in new real-time weather data.

In [12]:
# Fetching weather forecast measures for the next 5 days
weather_forecast_df = weather_measures.forecast_weather_measures(
    forecast_length=5
)

In [13]:
# Display the first 5 rows of the weather_forecast dataframe
weather_forecast_df.head(5)

,timestamp,datetime,date,hour,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1714694400000,2024-05-03 00:00:00,2024-05-03,0,14.3,65.0,0.0,0.0,0.0,1.0,25.0,20.5,36.0
1,1714698000000,2024-05-03 01:00:00,2024-05-03,1,13.6,69.0,0.0,0.0,0.0,0.0,12.0,21.6,37.4
2,1714701600000,2024-05-03 02:00:00,2024-05-03,2,13.0,72.0,0.0,0.0,0.0,0.0,7.0,20.9,37.4
3,1714705200000,2024-05-03 03:00:00,2024-05-03,3,12.7,73.0,0.0,0.0,0.0,1.0,26.0,19.8,34.6
4,1714708800000,2024-05-03 04:00:00,2024-05-03,4,12.4,73.0,0.0,0.0,0.0,2.0,54.0,18.7,33.8


In [14]:
# Display the last 5 rows of the weather_forecast dataframe
weather_forecast_df.tail(5)

,timestamp,datetime,date,hour,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
115,1715108400000,2024-05-07 19:00:00,2024-05-07,19,12.0,41.0,0.0,0.0,0.0,0.0,0.0,4.2,10.8
116,1715112000000,2024-05-07 20:00:00,2024-05-07,20,10.7,49.0,0.0,0.0,0.0,0.0,0.0,3.6,8.3
117,1715115600000,2024-05-07 21:00:00,2024-05-07,21,9.6,56.0,0.0,0.0,0.0,0.0,0.0,3.2,5.4
118,1715119200000,2024-05-07 22:00:00,2024-05-07,22,8.7,58.0,0.0,0.0,0.0,0.0,0.0,3.3,5.8
119,1715122800000,2024-05-07 23:00:00,2024-05-07,23,7.9,57.0,0.0,0.0,0.0,0.0,0.0,3.8,6.5


In [15]:
# Showing the information for the weather weather_forecast dataframe
weather_forecast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             120 non-null    int64         
 1   datetime              120 non-null    datetime64[ns]
 2   date                  120 non-null    object        
 3   hour                  120 non-null    int64         
 4   temperature_2m        120 non-null    float64       
 5   relative_humidity_2m  120 non-null    float64       
 6   precipitation         120 non-null    float64       
 7   rain                  120 non-null    float64       
 8   snowfall              120 non-null    float64       
 9   weather_code          120 non-null    float64       
 10  cloud_cover           120 non-null    float64       
 11  wind_speed_10m        120 non-null    float64       
 12  wind_gusts_10m        120 non-null    float64       
dtypes: datetime64[ns](1)

### <span style="color:#2656a3;"> 🗓️ Calendar of Danish workdays and holidays 
Lastly the calender data is being loaded in. The calendar data includes a 'type' attribute indicating whether the date is a holiday or not

In [16]:
calender_df = calendar.dk_calendar()

In [17]:
# Display the first 5 rows of the calender dataframe
calender_df.head(5)

,date,dayofweek,day,month,year,workday
0,2022-01-01,5,1,1,2022,0
1,2022-01-02,6,2,1,2022,0
2,2022-01-03,0,3,1,2022,1
3,2022-01-04,1,4,1,2022,1
4,2022-01-05,2,5,1,2022,1


In [18]:
# Display the last 5 rows of the calender dataframe
calender_df.tail(5)

,date,dayofweek,day,month,year,workday
1091,2024-12-27,4,27,12,2024,1
1092,2024-12-28,5,28,12,2024,0
1093,2024-12-29,6,29,12,2024,0
1094,2024-12-30,0,30,12,2024,1
1095,2024-12-31,1,31,12,2024,1


In [19]:
# Showing the information for the calender dataframe
calender_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       1096 non-null   object
 1   dayofweek  1096 non-null   int64 
 2   day        1096 non-null   int64 
 3   month      1096 non-null   int64 
 4   year       1096 non-null   int64 
 5   workday    1096 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 51.5+ KB


In [20]:
# Showing the information for the calender dataframe
calender_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       1096 non-null   object
 1   dayofweek  1096 non-null   int64 
 2   day        1096 non-null   int64 
 3   month      1096 non-null   int64 
 4   year       1096 non-null   int64 
 5   workday    1096 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 51.5+ KB


## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

We connect to Hopsworks Feature Store so we can access and create feature groups.

In [21]:
# Importing the hopsworks module
import hopsworks

# Logging in to the Hopsworks project
project = hopsworks.login()

# Getting the feature store from the project
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/554133
Connected. Call `.close()` to terminate connection gracefully.


### <span style="color:#2656a3;"> 🪄 Creating Feature Groups
A feature group can be seen as a collection of conceptually related features. In this case we create feature groups for the 
- eletricity price data,
- forecast_renewable_energy,
- weather data,
- calender data.

We specify a `primary_key` as `date`, so we are able to join them when we create a dataset for training later in part 03 the training_pipeline.
We define a name and a short describtion of the feature group's contents and a version number. 

`event_time` is specifyed as `timestamp`.

We've set `online_enabled` to `True` to enable accessing the feature group through the Online API for a Feature View.

In [22]:
# Creating the feature group for the electricity prices
electricity_fg = fs.get_or_create_feature_group(
    name="electricity_prices",
    version=1,
    description="Electricity prices from Energidata API",
    primary_key=["date","timestamp"], 
    online_enabled=True,
    event_time="timestamp",
)

We have now outlined metadata for the feature group. Data hasn't been stored yet, and there's no schema defined. To store data persistently for the feature group, we populate it with its associated data using the `insert` function.

In [23]:
# Inserting the electricity_df into the feature group named electricity_fg
electricity_fg.insert(electricity_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/554133/fs/549956/fg/778586


Uploading Dataframe: 0.00% |          | Rows 0/20469 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: electricity_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/electricity_prices_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x17f2dc950>, None)

We make a descriptions for each feature we put into the feature group. In this way we are adding more information and documentation to the user.

In [24]:
# List of descriptions for electricity features
electricity_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date of the electricity measurement"},
    {"name": "datetime", "description": "Date and time of the electricity measurement"},
    {"name": "hour", "description": "Hour of day"},
    {"name": "dk1_spotpricedkk_kwh", "description": "Spot price in DKK per KWH"}, 
]

# Updating feature descriptions
for desc in electricity_feature_descriptions: 
    electricity_fg.update_feature_description(desc["name"], desc["description"])

We replicate the process for both the `weather_fg` and `danish_holidays_fg` by establishing feature groups and inserting the dataframes into their respective feature groups.

In [25]:
# Creating the feature group for the weather data
weather_fg = fs.get_or_create_feature_group(
    name="weather_measurements",
    version=1,
    description="Weather measurements from Open Meteo API",
    primary_key=["date", "timestamp"], 
    event_time="timestamp",
    online_enabled=True,
)

In [26]:
# Inserting the weather_df into the feature group named weather_fg
weather_fg.insert(historical_weather_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/554133/fs/549956/fg/777575


Uploading Dataframe: 0.00% |          | Rows 0/20448 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_measurements_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/weather_measurements_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x308979690>, None)

In [27]:
# List of descriptions for weather features
weather_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the weather measurement"},
    {"name": "date", "description": "Date of the weather measurement"},
    {"name": "datetime", "description": "Date and time of the weather measurement"},
    {"name": "hour", "description": "Hour of day"},
    {"name": "temperature_2m", "description": "Temperature at 2m above ground"},
    {"name": "relative_humidity_2m", "description": "Relative humidity at 2m above ground"},
    {"name": "precipitation", "description": "Precipitation"},
    {"name": "rain", "description": "Rain"},
    {"name": "snowfall", "description": "Snowfall"},   
    {"name": "weather_code", "description": "Weather code"},   
    {"name": "cloud_cover", "description": "Cloud cover"},   
    {"name": "wind_speed_10m", "description": "Wind speed at 10m above ground"},   
    {"name": "wind_gusts_10m", "description": "Wind gusts at 10m above ground"},   
]

# Updating feature descriptions
for desc in weather_feature_descriptions: 
    weather_fg.update_feature_description(desc["name"], desc["description"])

In [28]:
# Creating the feature group for the danish calendar
danish_calendar_fg = fs.get_or_create_feature_group(
    name="dk_calendar",
    version=1,
    description="Danish calendar",
    online_enabled=True,
    primary_key=["date"],
)

In [29]:
# Inserting the calender_df into the feature group named danish_calendar_fg
danish_calendar_fg.insert(calender_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/554133/fs/549956/fg/777576


Uploading Dataframe: 0.00% |          | Rows 0/1096 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: dk_calendar_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/dk_calendar_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x3088ef590>, None)

In [31]:
# List of descriptions for danish_calendar features
danish_calendar_feature_descriptions = [
    {"name": "date", "description": "Date in the calendar"},
    {"name": "day", "description": "Day number of the week. Monday is 0 and Sunday is 6"},
    {"name": "month", "description": "Month number of the year"},
    {"name": "workday", "description": "Workday or not a workday"},
]

# Updating feature descriptions
for desc in danish_calendar_feature_descriptions: 
    danish_calendar_fg.update_feature_description(desc["name"], desc["description"])

---
## <span style="color:#2656a3;">⏭️ **Next:** Part 02: Feature Pipeline </span>

Next we will generate new data for the Feature Groups.